## **Import Utilities**

In [ ]:
import pandas as pd
import time
import anthropic
import sys
import os

sys.path.append('..')  # Add the parent directory of LLM_Evaluations to the Python path
from Utils.llm_evaluation_utils import load_responses_df, \
                        check_and_store_response,   \
                        build_question_prompt,      \
                        QUESTION_SETS

api_key = os.environ.get('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(api_key=api_key)

model_name = 'claude-3-sonnet-20240229'
max_tokens = 200        # Max generated tokens
temperature = 0.8
question_type = 'GS'
QUESTIONS = QUESTION_SETS[question_type]['QUESTIONS']

## **Claude Responses**

### Load Data

In [ ]:
# transcripts_dir = '../../Getting_Transcripts'
# transcripts_file_name = 'merged_filtered_videos_transcripts.csv'
# responses_dir = '../LLMs_Responses'
# topics_to_include = ['Spina Bifida', 'Flat Feet', 'Cluster Headache', 'Trigger Finger', 'Pudendal Nerve']

transcripts_dir = '../../../ISA_Paper/Data'
transcripts_file_name = 'diabetes_videos_transcripts.csv'
responses_dir = '../../../ISA_Paper/Data/Results'
topics_to_include = ['Insulin Self-Administration']

prompt_type = 'GS_prompting'
topics = 'diabetes'
results_file_name = f'{model_name}-{topics}-{prompt_type}'

responses_df = load_responses_df(transcripts_dir, transcripts_file_name, responses_dir, results_file_name, question_type)

print('responses_df shape:', responses_df.shape)
responses_df.head(2)

In [ ]:
if 'Topic' not in responses_df.columns:
    experts_file = '../../../Videos_and_DISCERN_data/filtered_experts_scores.csv'
    experts_df = pd.read_csv(experts_file)

    responses_df = responses_df.merge(experts_df[['Video ID', 'Topic']], on='Video ID', how='left')
    responses_df.insert(2, 'Topic', responses_df.pop('Topic'))
    responses_df = responses_df[responses_df['Topic'].isin(topics_to_include)]
    responses_df = responses_df.reset_index(drop=True)

print('responses_df shape:', responses_df.shape)
responses_df.head(2)

### Get Claude API Responses

In [ ]:
def query_model(client, model_name, prompt, max_tokens=250, temperature=0.8):
    '''Sends a prompt to the specified large language model and returns its response.'''
    response = client.messages.create(
        model=model_name,
        max_tokens=max_tokens,
        temperature=temperature,
        # system=system,
        messages=[
            {'role': 'user', 'content': prompt},
            {'role': 'assistant', 'content': 'Score:'}      # tell the LLM what to start with 
        ]
    )
    return response.content[0].text

print_response = False

# Calculate the delay based on your rate limit
requests_limit_per_minute = 50
delay = 60.0 / requests_limit_per_minute

for index, row in responses_df.iterrows():
    video_id = row['Video ID']
    transcript = row['Transcript']
    print(f'Started with video ID: {video_id} | Index: {index}')

    for question_num in range(1, len(QUESTIONS) + 1):
        column_name = f'Response_{question_num}'
        if row[column_name] == '':
            prompt = build_question_prompt(transcript, question_num, question_type)
            
            response = query_model(client, model_name, prompt, max_tokens, temperature)
            check_and_store_response(response, responses_df, video_id, question_num, 
                                     rating_scale=5, remove_prompt=False, print_response=print_response)
            
            time.sleep(delay)

### Explore Results

In [ ]:
display_from_index = 0
index_of_q1 = responses_df.columns.get_loc("Q1")

responses_df.iloc[display_from_index:, index_of_q1:index_of_q1+15].head()

In [ ]:
columns_with_none = (responses_df.isna() | (responses_df == '')).sum()
columns_with_none

In [ ]:
rows_with_none = responses_df[responses_df.isna().any(axis=1)]
rows_with_none

In [ ]:
indices_with_problems = responses_df[responses_df['Problem'].apply(lambda x: len(x) > 0)].index.tolist()
print(indices_with_problems)

In [ ]:
from IPython.display import display, HTML

if indices_with_problems:
    index_with_problem = 11
    responses_with_problem_list = list(responses_df.loc[index_with_problem, 'Problem'])
    print("List of questions with problem:", responses_with_problem_list)

    response_with_problem = responses_with_problem_list[0]
    text = responses_df.loc[index_with_problem, f'Response_{response_with_problem}']
    display(HTML("<div style='white-space: pre-wrap;'>{}</div>".format(text)))

In [ ]:
# display the full responses for a specific transcript
index_to_display = 45
for question_num in range(1, 16):
    print(f'Q{question_num}:', responses_df.at[index_to_display,f'Response_{question_num}'])

### Store Results in a CSV File

In [ ]:
csv_output_file = os.path.join(responses_dir, f'{results_file_name}-response.csv')

responses_df.to_csv(csv_output_file, index=False, encoding='utf-8')